### №1. Начало работы с БД ClickHouse
В этом ноутбуке я создам базу данных ClickHouse в Docker контейнере. После этого нужно будет подключиться к БД, создать таблицу, занести тестовые данные из файла sm_data.csv в эту таблицу. Таким образом, у нас будет табличка с данными, которую можно будет обрабатывать в дальнейшем

In [2]:
"Создаем базу данных в ClickHouse"
try:
    import re
    from clickhouse_driver import Client

    query_db = '''
    CREATE DATABASE IF NOT EXISTS db_test ENGINE = Memory
    '''

    client = Client(host='localhost', port=9000, user='default', password='pivanet')

    client.execute(query_db)
    print('База данных успешно создана')
except Exception as e:
    print(f'Что-то пошло не так: {e}')

[]

In [4]:
"Создаем таблицу в БД"

from clickhouse_driver import Client

try:
# SQL query to create a table
    query_create_table = '''
    CREATE TABLE IF NOT EXISTS db_test.logss (
        timestamp DateTime64(3),
        level String,
        sys String,
        mrf String,
        user String,
        script_id String,
        script_name String,
        script_key String,
        script_version String,
        script_process String,
        action String,
        action_entity String,
        action_id String,
        action_data String,
        message String,
        script_step_id String,
        script_step_name String,
        current_script_name String,
        current_script_key String,
        current_script_version String,
        block_type String,
        parameters String,
        timezone String,
        communication_id String,
        communication_number String,
        integration_service_id String,
        duration Float32,
        client_mrf String,
        session String,
        script_owner String,
        current_script_owner String,
        script_responsible String,
        current_script_responsible String,
        crm_departament String
    ) ENGINE = MergeTree()
    ORDER BY timezone
    '''
    client = Client(host='localhost', port=9000, user='default', password='pivanet')

    client.execute(query_create_table)
    print('Таблица успешно создана')
except Exception as e:
    print(f'Что-то пошло не так: {e}')

[]

In [ ]:
"Статичные переменные"
local_file_path = 'sm_data.csv'
container_name = 'clickhouse-server_2'

container_path = '/tmp/sm_data.csv'

In [5]:
"Скопируем файл в окружение Docker"
import subprocess


command = ['docker', 'cp', local_file_path, f'{container_name}:{container_path}']

result = subprocess.run(command, capture_output=True, text=True)

if result.returncode == 0:
    print("Файл успешно скопирован.")
else:
    print("Ошибка копирования файла:", result.stderr)

Файл успешно скопирован.


In [7]:
"Проверим, что файл попал в окружение Docker"

command_to_run = 'ls /tmp'
command = ['docker', 'exec', container_name] + command_to_run.split()
result = subprocess.run(command, capture_output=True, text=True)
if result.returncode == 0:
    print("Вывод команды:")
    print(result.stdout)
else:
    print("Ошибка выполнения команды:", result.stderr)

Вывод команды:
bitnami
sm_data.csv



In [9]:
"Загрузим файл в БД ClickHouse"

bash_command = f"clickhouse-client --user default --password pivanet --query='INSERT INTO db_test.logss FORMAT CSV' < {container_path}"
command = ['docker', 'exec', '-i', container_name, 'bash', '-c', bash_command]
result = subprocess.run(command, capture_output=True, text=True)
if result.returncode == 0:
    print("Команда выполнена успешно.")
else:
    print("Ошибка выполнения команды:", result.stderr)

Команда выполнена успешно.
